In [1]:
import psycopg2
import urllib
import json
import velovUtils
import pandas as pd
from datetime import timedelta
import datetime
import seaborn as sns
from scipy.stats import exponweib
url = 'https://download.data.grandlyon.com/ws/rdata/jcd_jcdecaux.jcdvelov/all.json'
conn = psycopg2.connect(database = "velov", 
                        user = "dev", 
                        password = "velov", 
                        host = "localhost" ,
                        port = "5432")

In [2]:
sqlDropStationsData = "DROP TABLE stations_data"
sqlDropStations = "DROP TABLE stations"
sqlDropStationsHours = "DROP TABLE stations_hours"

sqlCreateStationsData = "CREATE TABLE stations_data ( station_id integer, available_bikes integer, last_update timestamp without time zone, last_update_fme timestamp without time zone, status character varying(20), available_bike_stands integer, availabilitycode integer )"
sqlCreateStations = "CREATE TABLE stations (station_id integer, name character varying(50), number integer, lat integer, lng integer, bike_stands integer, address character varying(50), code_insee integer, commune character varying(20))"
sqlCreateStationsHours = "CREATE TABLE stations_hours (station_id integer, hour integer, exponentiation numeric, shape numeric, loc numeric, scale numeric)"

In [ ]:
cur = conn.cursor()
cur.execute(sqlDropStations)
cur.execute(sqlDropStationsData)

conn.commit()

cur = conn.cursor()
cur.execute(sqlCreateStationsHours)
conn.commit()

In [ ]:
# Create sql statement for retrieve data from database for given station and given hour
station = 11
hour = 13
sqlSelect = "select * from stations_data where station_id =" + str(
        station) + " and extract(HOUR FROM last_update_fme) =" + str(hour) + " order by last_update"
cur = conn.cursor()
cur.execute(sqlSelect)
data = cur.fetchall()
dataAsDF = pd.DataFrame(data)
dataAsDF

In [ ]:
# Create sql statement for retrieve data from database for given station and given hour
station = 11
hour = 9
sqlSelect = "select distinct last_update, available_bike_stands last_update_fme from velovdata where station_name =" + str(
        station) + " and extract(HOUR FROM last_update_fme) =" + str(hour) + " order by last_update"

In [ ]:
# Retrieve all data for station at given hour into a pandas.dataFrame
cur = conn.cursor()
cur.execute(sqlSelect)
data = cur.fetchall()
dataAsDF = pd.DataFrame(data)
labels = ['last_update', 'avail_bike_stands', "last_update_fme"]
for columns_label in range(0, len(labels)):
    dataAsDF = dataAsDF.rename(columns={columns_label: labels[columns_label]})

In [ ]:
dataAsDF

In [ ]:
# First add to dataAsDF column for duration between two update
duration = [timedelta(seconds=0)] + list(dataAsDF.last_update[i + 1] - dataAsDF.last_update[i] 
                                         for i in range(0, len(dataAsDF) - 1))
dataAsDF = pd.concat([dataAsDF, pd.DataFrame(duration)], axis=1)
dataAsDF = dataAsDF.rename(columns={0: 'duration_since_last_update'})

In [ ]:
dataAsDF

In [ ]:
dataAsDF['real_update'] = dataAsDF['duration_since_last_update'].apply(lambda update: 
                                                                       update <= timedelta(minutes=10))

In [ ]:
dataAsDF.to_csv('test.csv')

In [ ]:
sns.displot(data=dataAsDF.loc[dataAsDF['real_update']==True]['avail_bike_stands'],
            kde=True)

In [ ]:
weibulltheo = exponweib.rvs(1, 0.5, loc=0, scale=1, size=200)
sns.displot(data=weibulltheo,
           kde=True)